In [1]:
import os

assert os.environ['CONDA_DEFAULT_ENV']=='ttv'

## TOIs

In [2]:
import sys
sys.path.insert(0, '../code')
from utils import *

In [4]:
df = get_tois(clobber=False) #set to True only every new TOI releases
df.tail()

Saved:  TOIs with TFPWG disposition==FP are removed.


,TIC ID,TOI,Previous CTOI,Master,SG1A,SG1B,SG2,SG3,SG4,SG5,...,Stellar Radius (R_Sun) err,Stellar Metallicity,Stellar Metallicity err,Stellar Mass (M_Sun),Stellar Mass (M_Sun) err,Sectors,Date TOI Alerted (UTC),Date TOI Updated (UTC),Date Modified,Comments
7142,427765233,6973.01,TIC 427765233.01,1,4,1,1,1,4,4,...,0.05,NaN,NaN,1.000,0.124793,"17,18,24,57,58",2024-04-25,2024-04-25,2024-04-30 12:16:05,L1 candidate
7143,348684814,6974.01,NaN,3,4,3,3,1,4,4,...,0.07,NaN,NaN,1.026,0.128427,"8,35,61",2024-04-25,2024-04-25,2024-04-30 12:16:05,TFOP determined this star (TIC 348684814) is t...
7144,158381418,6975.01,NaN,3,5,5,3,1,4,4,...,NaN,NaN,NaN,NaN,NaN,"9,36,63",2024-04-25,2024-04-25,2024-04-30 12:16:05,TIC 941996822 0.55'' away; this is the brighte...
7145,5972036,6976.01,NaN,3,4,3,3,1,4,4,...,0.07,NaN,NaN,1.480,0.249393,"9,35,36,62,63",2024-04-25,2024-04-25,2024-04-30 12:16:05,transits deeper in s62 and s63; Rp slightly un...
7146,436575927,6977.01,NaN,5,5,5,5,5,5,5,...,0.06,NaN,NaN,0.660,0.080628,"43,44,70,71",2024-04-25,2024-04-25,2024-04-30 12:16:05,K2-394 b


In [5]:
tics = df['TIC ID'].unique()
len(tics)

5856

In [6]:
len(df['TOI'].unique())

6120

In [7]:
nsys = df['TIC ID'].value_counts()
nsys

TIC ID
425997655    5
367630162    5
142276270    4
150428135    4
260647166    4
            ..
156949982    1
409896170    1
231337271    1
157011920    1
436575927    1
Name: count, Length: 5856, dtype: int64

In [8]:
tics_multis = list(nsys[nsys>1].index)
print(len(tics_multis))

206


## TOI 1136 (Dai+2023)
First order MMR: 3:2, 2:1, 3:2, 7:5, and 3:2

In [9]:
col = 'Period (days)'
d = df[df['TIC ID']==142276270].sort_values(by=col)
d[col]

1131     6.258792
1130    12.518728
1133    18.804756
1132    26.317954
Name: Period (days), dtype: float64

In [10]:
# add two outer planets
periods = [4.17278, 6.25725, 12.51937, 18.7992, 26.3162, 39.5387]
periods

[4.17278, 6.25725, 12.51937, 18.7992, 26.3162, 39.5387]

https://arxiv.org/abs/1208.3312

$$
\frac{p'}{p} \sim \frac{j}{j-1}
$$

proximity to resonance:
$$
\Delta = \frac{p'}{p} \frac{j-1}{j} - 1
$$

In [11]:
get_orbit_pairs(10, order=1)

[(1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9)]

In [12]:
resonances_1st_order, deltas = get_resonant_pairs(periods, order=1, tol=0.01)
resonances_1st_order, deltas

(['3:2 | P=(4.17,6.26) (delta=0.03%)',
  '2:1 | P=(6.26,12.52) (delta=0.04%)',
  '3:2 | P=(12.52,18.80) (delta=0.11%)',
  '3:2 | P=(26.32,39.54) (delta=0.16%)'],
 [0.0003067499364933779,
  0.0003891485876383882,
  0.001072737685682057,
  0.0016314412161837488])

In [13]:
resonances_2nd_order, deltas = get_resonant_pairs(periods, order=2, tol=0.01)
resonances_2nd_order, deltas

(['6:4 | P=(4.17,6.26) (delta=0.03%)',
  '4:2 | P=(6.26,12.52) (delta=0.04%)',
  '6:4 | P=(12.52,18.80) (delta=0.11%)',
  '7:5 | P=(18.80,26.32) (delta=0.01%)',
  '6:4 | P=(26.32,39.54) (delta=0.16%)'],
 [0.0003067499364933779,
  0.0003891485876383882,
  0.001072737685682057,
  0.00010182804131475542,
  0.0016314412161837488])

In [14]:
from tess_stars2px import tess_stars2px_function_entry

ticid = 231663901
data_json = get_tfop_info(f"TIC {ticid}")
ra = float(data_json['coordinates']['ra'])
dec = float(data_json['coordinates']['dec'])
outID, outEclipLong, outEclipLat, outSec, outCam, outCcd, \
            outColPix, outRowPix, scinfo = tess_stars2px_function_entry(
                    ticid, ra, dec)
outSec

array([ 1, 27, 67, 94, 95])

## All TOIs

In [15]:
from tqdm.notebook import tqdm

ttv_1st_order = []       # commensurability in 1 planet pair 
ttv_2nd_order = []
ttv_chain = []           # more than 1 planet pair
deltas1 = []
deltas2 = []
texts = []
sectors = []
for tic in tqdm(tics_multis):
    dd = df[df['TIC ID']==tic].squeeze()
    periods = dd["Period (days)"].values
    Nplanets = len(periods)
    toi = str(dd.iloc[0].TOI).split('.')[0]
    
    resonances_1st_order, delta1 = get_resonant_pairs(periods, order=1, tol=0.01);
    resonances_2nd_order, delta2 = get_resonant_pairs(periods, order=2, tol=0.01);
    msg = f"TOI-{str(toi).zfill(4)} | [TIC-{tic}](https://exofop.ipac.caltech.edu/tess/target.php?id={tic}) (nplanets={Nplanets}):<br>\n"
    show = False
    if len(resonances_1st_order)>=1:
        msg += f"First Order: {' '.join(resonances_1st_order)}<br>\n"
        ttv_1st_order.append(tic)
        deltas1.append(min(delta1))
        if len(resonances_1st_order)>=2:
            ttv_chain.append(tic)         
        show = True
    if len(resonances_2nd_order)>=1:
        if len(resonances_1st_order)>=1:
            i = int(resonances_1st_order[0].split(':')[0])
            j = int(resonances_2nd_order[0].split(':')[0])
            if j/i!=2:
                msg += f"Second Order: {' '.join(resonances_2nd_order)}\n"
                ttv_2nd_order.append(tic)
                deltas2.append(min(delta2))
                show = True
        else:
            msg += f"Second Order: {' '.join(resonances_2nd_order)}\n"
            ttv_2nd_order.append(tic)
            deltas2.append(min(delta2))
            show = True

    # check if observed by TESS
    if tic is not None:
        data_json = get_tfop_info(f"TIC {tic}")
        ra = float(data_json['coordinates']['ra'])
        dec = float(data_json['coordinates']['dec'])
        outID, outEclipLong, outEclipLat, outSec, outCam, outCcd, \
                    outColPix, outRowPix, scinfo = tess_stars2px_function_entry(
                            tic, ra, dec)
        if len(outSec)>0:
            msg += f"TESS sectors={outSec}<br>\n"
            
    if show:
        print(msg)
        texts.append(msg)

  0%|          | 0/206 [00:00<?, ?it/s]

TOI-0174 | [TIC-425997655](https://exofop.ipac.caltech.edu/tess/target.php?id=425997655) (nplanets=5):<br>
First Order: 2:1 | P=(3.98,7.91) (delta=0.58%)<br>
TESS sectors=[ 1  2  3  4 11 29 30 31 34 64 68 69 95 96]<br>

TOI-2104 | [TIC-367630162](https://exofop.ipac.caltech.edu/tess/target.php?id=367630162) (nplanets=5):<br>
Second Order: 5:3 | P=(1.98,3.27) (delta=0.85%)
TESS sectors=[14 19 20 26 40 47 53 73 74]<br>

TOI-1136 | [TIC-142276270](https://exofop.ipac.caltech.edu/tess/target.php?id=142276270) (nplanets=4):<br>
First Order: 2:1 | P=(6.26,12.52) (delta=0.01%) 3:2 | P=(12.52,18.80) (delta=0.14%)<br>
TESS sectors=[14 15 21 22 41 48 75]<br>

TOI-0700 | [TIC-150428135](https://exofop.ipac.caltech.edu/tess/target.php?id=150428135) (nplanets=4):<br>
First Order: 4:3 | P=(27.81,37.42) (delta=0.93%)<br>
TESS sectors=[ 1  3  4  5  6  7  8  9 10 11 13 27 28 30 31 33 34 35 36 37 38 61 62 63
 64 65 66 68 69 87 88 89 90 93 95 96]<br>

TOI-0282 | [TIC-29781292](https://exofop.ipac.caltech

/Users/jpdl/github/research/project/young_ttvs/notebooks/../code/utils.py:317: RuntimeWarning: divide by zero encountered in double_scalars
  delta = abs((Pout/Pin)*(i/j) - 1)


TOI-0175 | [TIC-307210830](https://exofop.ipac.caltech.edu/tess/target.php?id=307210830) (nplanets=3):<br>
First Order: 2:1 | P=(3.69,7.45) (delta=0.94%)<br>
TESS sectors=[ 2  5  8  9 10 11 12 28 29 32 35 36 37 38 39 61 62 63 64 65 69 87 88 89
 90 96]<br>

TOI-1730 | [TIC-318022259](https://exofop.ipac.caltech.edu/tess/target.php?id=318022259) (nplanets=3):<br>
First Order: 2:1 | P=(6.23,12.57) (delta=0.98%)<br>
TESS sectors=[20 47 60]<br>

TOI-1749 | [TIC-233602827](https://exofop.ipac.caltech.edu/tess/target.php?id=233602827) (nplanets=3):<br>
First Order: 2:1 | P=(4.49,9.04) (delta=0.74%)<br>
TESS sectors=[14 15 16 17 18 19 20 21 23 24 25 26 40 41 47 48 49 50 51 52 53 54 55 57
 58 59 60 74 77 78 79 80 81 82 84 85 86]<br>

TOI-1670 | [TIC-441739020](https://exofop.ipac.caltech.edu/tess/target.php?id=441739020) (nplanets=3):<br>
Second Order: 3:1 | P=(40.75,123.06) (delta=0.66%)
TESS sectors=[14 15 16 18 19 20 21 22 23 24 25 26 40 41 47 48 49 50 51 52 53 55 56 57
 58 59 60 73 74 75 76

In [16]:
len(ttv_1st_order), len(ttv_2nd_order)

(22, 11)

In [17]:
import pandas as pd

s1 = pd.DataFrame(np.c_[ttv_1st_order,deltas1], columns=['tic','delta_min'])
s1['1st order'] = True
s1.sort_values(by='delta_min')

,tic,delta_min,1st order
7,441798995.0,0.000023,True
1,142276270.0,0.000091,True
8,459837008.0,0.000158,True
15,384984325.0,0.000488,True
17,455947620.0,0.001917,True
13,143022742.0,0.002051,True
18,303204165.0,0.002446,True
3,29781292.0,0.003024,True
14,153065527.0,0.004089,True
11,259377017.0,0.005161,True


In [18]:
s2 = pd.DataFrame(np.c_[ttv_2nd_order,deltas2], columns=['tic','delta_min'])
s2['2nd order'] = True
s2.sort_values(by='delta_min')

,tic,delta_min,2nd order
3,178155732.0,0.000263,True
9,294471966.0,0.001432,True
2,27491137.0,0.002862,True
10,17129975.0,0.003054,True
1,441739020.0,0.006611,True
4,468979441.0,0.006748,True
5,397362481.0,0.007041,True
6,167342439.0,0.007097,True
7,392681545.0,0.007379,True
0,367630162.0,0.008548,True


In [19]:
res = pd.merge(s1, s2, how='outer')
res = res.replace(np.nan, False)
res['tic'] = res.tic.map(int)
res = res.sort_values(by='delta_min')
res.head()

/var/folders/4s/07r41mpn76n_0fykwqyjgc840000gn/T/ipykernel_11223/947716635.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  res = res.replace(np.nan, False)


,tic,delta_min,1st order,2nd order
29,441798995,0.000023,True,False
7,142276270,0.000091,True,False
31,459837008,0.000158,True,False
12,178155732,0.000263,False,True
24,384984325,0.000488,True,False


In [20]:
len(res)

33

In [21]:
len(texts), len(res)

(33, 33)

In [22]:
for tic in res.tic:
    idx = [True if i.split('TIC-')[1].split(']')[0]==str(tic) else False for i in texts]
    x = np.array(texts)[idx]
    print(x[0] if len(x)>0 else '')

TOI-2269 | [TIC-441798995](https://exofop.ipac.caltech.edu/tess/target.php?id=441798995) (nplanets=3):<br>
First Order: 2:1 | P=(1.42,2.84) (delta=0.00%)<br>
TESS sectors=[14 15 16 17 18 19 20 21 22 23 24 25 26 40 41 47 48 49 50 51 52 53 54 55
 56 57 58 59 60 73 74 75 76 77 78 79 80 81 82 83 84 85 86]<br>

TOI-1136 | [TIC-142276270](https://exofop.ipac.caltech.edu/tess/target.php?id=142276270) (nplanets=4):<br>
First Order: 2:1 | P=(6.26,12.52) (delta=0.01%) 3:2 | P=(12.52,18.80) (delta=0.14%)<br>
TESS sectors=[14 15 21 22 41 48 75]<br>

TOI-2267 | [TIC-459837008](https://exofop.ipac.caltech.edu/tess/target.php?id=459837008) (nplanets=3):<br>
First Order: 9:8 | P=(2.03,2.29) (delta=0.02%)<br>
TESS sectors=[19 20 25 26 40 52 53 59 60 73 79 86]<br>

TOI-0396 | [TIC-178155732](https://exofop.ipac.caltech.edu/tess/target.php?id=178155732) (nplanets=3):<br>
Second Order: 5:3 | P=(3.59,5.97) (delta=0.03%)
TESS sectors=[ 3  4 30 31]<br>

TOI-6109 | [TIC-384984325](https://exofop.ipac.caltech.

In [23]:
res.to_csv('../data/tics_resonance_from_toi.csv')

In [24]:
idx = df['TIC ID'].isin(res.tic)
tois_in_resonance = df['TOI'][idx].apply(lambda x: str(x).split('.')[0].zfill(4)).unique()
tois_in_resonance

array(['0174', '0175', '0216', '0270', '0282', '0396', '0406', '0663',
       '0696', '0700', '0707', '1097', '1136', '1339', '1446', '1670',
       '1730', '1749', '2076', '2104', '2267', '2269', '4323', '4495',
       '4647', '5493', '5520', '6022', '6054', '6109', '6265', '6276',
       '6969'], dtype=object)

In [25]:
np.savetxt('../data/tics_resonance_from_toi.txt', res.tic.tolist(), fmt="%d")
np.savetxt('../data/tois_resonance_from_toi.txt', tois_in_resonance, fmt="%s")
np.savetxt('../data/tics_resonance_from_toi_1st_order.txt', ttv_1st_order, fmt="%d")
np.savetxt('../data/tics_resonance_from_toi_2nd_order.txt', ttv_2nd_order, fmt="%d")

## Transit timing database
* https://ui.adsabs.harvard.edu/abs/2022ApJS..259...62I/abstract

In [26]:
#https://github.com/transit-timing/tt/blob/master/3_database/table4.csv
url = "https://raw.githubusercontent.com/transit-timing/tt/master/3_database/table4.csv"
df2 = pd.read_csv(url)
df2

,System,Orbit number,T_mid,Uncertainty (days),Time System,#,Reference
0,CoRoT-01,-1412,2.454138e+06,0.000470,BJD,1,2009A&A...506..369B
1,CoRoT-01,-1411,2.454140e+06,0.000590,BJD,1,2009A&A...506..369B
2,CoRoT-01,-1410,2.454141e+06,0.000620,BJD,1,2009A&A...506..369B
3,CoRoT-01,-1409,2.454143e+06,0.000390,BJD,1,2009A&A...506..369B
4,CoRoT-01,-1408,2.454144e+06,0.001630,BJD,1,2009A&A...506..369B
...,...,...,...,...,...,...,...
8662,XO-7,214,2.459407e+06,0.000510,BJD_TDB,1,This work
8663,XO-7,215,2.459410e+06,0.000467,BJD_TDB,1,This work
8664,XO-7,216,2.459413e+06,0.000508,BJD_TDB,1,This work
8665,XO-7,217,2.459415e+06,0.000517,BJD_TDB,1,This work
